## Real-time Bitcoin Analysis with PySparkling

In [1]:
pip install h2o_pysparkling_2.3



[notice] A new release of pip is available: 24.3.1 -> 25.1
[notice] To update, run: /Users/saanika/Saanika Patil/saanikap/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pyspark h2o pysparkling requests pandas matplotlib seaborn findspark



[notice] A new release of pip is available: 24.3.1 -> 25.1
[notice] To update, run: /Users/saanika/Saanika Patil/saanikap/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession

# Initialize Spark session
spark = SparkSession.builder.master("local[*]").appName("BitcoinAnalysis").getOrCreate()

# Check the Spark version
print(spark.version)



25/04/30 17:59:52 WARN Utils: Your hostname, SAANIKAs-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 10.0.0.79 instead (on interface en0)
25/04/30 17:59:52 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/30 17:59:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/04/30 17:59:53 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


3.5.5


In [4]:
pip install requests pandas



[notice] A new release of pip is available: 24.3.1 -> 25.1
[notice] To update, run: /Users/saanika/Saanika Patil/saanikap/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
import requests
import pandas as pd

# Function to fetch Bitcoin price from CoinGecko
def fetch_bitcoin_price():
    url = 'https://api.coingecko.com/api/v3/simple/price?ids=bitcoin&vs_currencies=usd'
    response = requests.get(url)
    data = response.json()
    return data['bitcoin']['usd']

# Fetch the latest Bitcoin price
bitcoin_price = fetch_bitcoin_price()
print(f"Bitcoin price in USD: {bitcoin_price}")


Bitcoin price in USD: 94489


In [6]:
from pyspark.sql import SparkSession
from pyspark.streaming import StreamingContext

# Initialize Spark session
spark = SparkSession.builder.master("local[*]").appName("BitcoinAnalysis").getOrCreate()

# Create a streaming context with a batch interval of 10 seconds
ssc = StreamingContext(spark.sparkContext, 10)  # 10-second batch interval



/Users/saanika/Downloads/spark-3.5.5-bin-hadoop3/python/pyspark/streaming/context.py:72: FutureWarning: DStream is deprecated as of Spark 3.4.0. Migrate to Structured Streaming.
  warnings.warn(


+----------+-------+
| timestamp|  price|
+----------+-------+
|      NULL|   NULL|
|1633036800|43000.0|
|1633036900|43200.0|
|1633037000|43400.0|
|1633037100|43600.0|
|1633037200|43800.0|
+----------+-------+



In [7]:
from pyspark.sql.types import StructType, StructField, LongType, DoubleType

# Define the schema for the Bitcoin price data
schema = StructType([
    StructField("timestamp", LongType(), True),
    StructField("price", DoubleType(), True)
])


In [8]:
import time

# Initialize Spark session
spark = SparkSession.builder.master("local[*]").appName("BitcoinAnalysis").getOrCreate()

# Define the schema for the Bitcoin price data
schema = StructType([
    StructField("timestamp", LongType(), True),
    StructField("price", DoubleType(), True)
])

# Create a streaming DataFrame using the correct path to the directory
bitcoin_stream = spark.readStream.schema(schema).csv("/Users/saanika/Saanika Patil/DATA605/bitcoin_data")

# Perform any data processing here (e.g., cleaning, transformations, etc.)
bitcoin_stream = bitcoin_stream.withColumn("price", bitcoin_stream["price"])

# Define the query that will continuously process the data
query = bitcoin_stream.writeStream \
    .outputMode("append") \
    .format("console") \
    .start()

# Wait for the termination of the stream for a specific time
time.sleep(60)  # Run the stream for 60 seconds
query.stop()  # Stop the stream after 60 seconds


25/04/30 17:59:55 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/3f/s6ws9m9j6p13sb35t40r0q1c0000gn/T/temporary-819a8db1-9997-44c9-a4b8-724b1768b66f. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/04/30 17:59:55 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+----------+-------+
| timestamp|  price|
+----------+-------+
|      NULL|   NULL|
|1633036800|43000.0|
|1633036900|43200.0|
|1633037000|43400.0|
|1633037100|43600.0|
|1633037200|43800.0|
+----------+-------+



In [9]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, LongType, DoubleType

# Define the process_batch function to handle each batch of data
def process_batch(df, epoch_id):
    # Process each batch here
    df.show()  # Just display the batch data, you can replace this with your processing logic
    # You can also perform additional transformations, aggregations, or save the data

# Initialize Spark session
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("BitcoinAnalysis") \
    .config("spark.sql.shuffle.partitions", "2") \
    .config("spark.streaming.stopGracefullyOnShutdown", "true") \
    .getOrCreate()

# Define the schema for the Bitcoin price data
schema = StructType([
    StructField("timestamp", LongType(), True),
    StructField("price", DoubleType(), True)
])

# Create a streaming DataFrame using the correct path to the directory
bitcoin_stream = spark.readStream \
    .schema(schema) \
    .csv("/Users/saanika/Saanika Patil/DATA605/bitcoin_data")

# Perform any data processing here (e.g., cleaning, transformations, etc.)
bitcoin_stream = bitcoin_stream.withColumn("price", bitcoin_stream["price"])

# Define the query with foreachBatch processing
query = bitcoin_stream.writeStream \
    .foreachBatch(process_batch) \
    .outputMode("append") \
    .option("checkpointLocation", "/Users/saanika/Saanika Patil/DATA605/bitcoin_data/checkpoint") \
    .start()

# Wait for the termination of the stream
query.awaitTermination()


25/04/30 18:00:55 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.
25/04/30 18:00:55 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
25/04/30 18:06:20 WARN FileStreamSource: Listed 1 file(s) in 4354 ms
25/04/30 23:41:40 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 1629464 ms exceeds timeout 120000 ms
25/04/30 23:41:40 WARN SparkContext: Killing executors is not supported by current scheduler.
25/04/30 23:41:40 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv

Py4JError: An error occurred while calling o65.awaitTermination

In [ ]:
# Create a streaming DataFrame with a limit on files per trigger
bitcoin_stream = spark.readStream.schema(schema).option("maxFilesPerTrigger", 1).csv("/Users/saanika/Saanika Patil/DATA605/bitcoin_data")

# Define the query
query = bitcoin_stream.writeStream \
    .outputMode("append") \
    .format("console") \
    .start()

query.awaitTermination()


In [ ]:
from h2o.estimators import H2OGradientBoostingEstimator
import h2o

h2o.init()

# Convert pandas DataFrame to H2O frame
train_data = h2o.H2OFrame(pandas_df)

# Define the model
model = H2OGradientBoostingEstimator()
model.train(x=["timestamp"], y="price", training_frame=train_data)

# Evaluate the model
performance = model.model_performance(train_data)
print(performance)


In [ ]:
import matplotlib.pyplot as plt

# Plot the data
def plot_data(data):
    plt.plot(data['timestamp'], data['price'])
    plt.xlabel("Time")
    plt.ylabel("Price (USD)")
    plt.title("Real-time Bitcoin Price")
    plt.show()
